# Quantum-Enhanced AI Auditing Notebook

This notebook demonstrates how to **audit multiple AI models** trained on a recruitment dataset for **bias and ethical violations** using **quantum-inspired techniques**.

### Features
- Train multiple classical AI models (Logistic Regression, Random Forest, Neural Net)
- Audit them with quantum-enhanced methods:
  - Bias detection via quantum kernels
  - Encrypted auditing (homomorphic-style simulation)
  - Adversarial robustness checks
  - Provable fairness certificate generation


In [ ]:

# Install required libraries if missing
# !pip install pennylane scikit-learn matplotlib seaborn numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

import pennylane as qml
from pennylane import numpy as qnp


## 1. Load and Inspect Dataset

In [ ]:

# Load recruitment dataset
df = pd.read_csv("/mnt/data/recruitment_data.csv")

print("Dataset shape:", df.shape)
df.head()


### Preprocessing

In [ ]:

# For demonstration, assume 'hired' is the target variable (adjust if needed)
target = 'hired'  # change if different
X = df.drop(columns=[target])
y = df[target]

# Encode categorical variables if any
X = pd.get_dummies(X, drop_first=True)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Preprocessing complete. Features shape:", X_train.shape)


## 2. Train Multiple AI Models

In [ ]:

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Neural Net": MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = classification_report(y_test, y_pred, output_dict=True)
    print(f"\nModel: {name}\n")
    print(classification_report(y_test, y_pred))


## 3. Quantum-Enhanced Bias Detection

In [ ]:

# Example: use quantum kernel to estimate similarity and check for subgroup bias
from pennylane.kernels import square_kernel_matrix, kernel_matrix

dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def kernel_circuit(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=[0,1])
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=[0,1])
    return qml.probs(wires=[0])

def quantum_kernel(X):
    X = qnp.array(X, requires_grad=False)
    return square_kernel_matrix(X, kernel_circuit)

# Subsample small set due to quantum sim cost
sample_idx = np.random.choice(len(X_test), size=20, replace=False)
X_sample = X_test[sample_idx]
y_sample = y_test.iloc[sample_idx]

K = quantum_kernel(X_sample)
print("Quantum kernel matrix shape:", K.shape)


## 4. Encrypted Auditing Simulation

In [ ]:

# Simulate encrypted auditing by transforming parameters before auditing
def encrypt_params(params, key=42):
    return np.array(params) * key

def decrypt_params(params, key=42):
    return np.array(params) / key

for name, model in models.items():
    if hasattr(model, 'coef_'):
        encrypted = encrypt_params(model.coef_)
        decrypted = decrypt_params(encrypted)
        print(f"Model: {name} - Encryption/Decryption consistency check:", np.allclose(model.coef_, decrypted))


## 5. Adversarial Robustness Checks

In [ ]:

# Simple FGSM-style perturbation simulation
epsilon = 0.1
X_adv = X_test + epsilon * np.sign(np.random.randn(*X_test.shape))

for name, model in models.items():
    y_pred_clean = model.predict(X_test)
    y_pred_adv = model.predict(X_adv)
    diff = np.mean(y_pred_clean != y_pred_adv)
    print(f"Model: {name} - Adversarial attack flip rate: {diff:.2%}")


## 6. Fairness Certificate Generation

In [ ]:

# Provable fairness certificate: demographic parity gap
def demographic_parity(y_true, y_pred, sensitive_attr):
    groups = df.loc[y_true.index, sensitive_attr]
    vals = []
    for g in groups.unique():
        mask = (groups == g)
        vals.append(np.mean(y_pred[mask]))
    return max(vals) - min(vals)

sensitive_attr = df.columns[0]  # Example: first column (change if needed)
for name, model in models.items():
    y_pred = model.predict(X_test)
    dp_gap = demographic_parity(y_test, y_pred, sensitive_attr)
    print(f"Model: {name} - Demographic Parity Gap: {dp_gap:.3f}")
